In [4]:
import polars as pl
from scipy.interpolate import CubicSpline
from jinja2 import Template

In [5]:
most_data = pl.read_csv("data/LandColor.csv")
forest_data = pl.read_csv("data/LandColorForest.csv")
data = most_data.join(forest_data.select("shapeName", "date", pl.selectors.starts_with("forest_")), on=["shapeName", "date"])

In [6]:
long = (data
    .select(pl.all().exclude([".geo", "system:index"]))
    .unpivot(index = ["date", "shapeGroup", "shapeName", "shapeType"], variable_name = "variable", value_name="meanReflectance")
    .with_columns(
        pl.from_epoch(pl.col("date"), time_unit="ms"),
        pl.col("variable").str.splitn('_',2).struct.rename_fields(["landCover","band"])
    )
    .with_columns(month = pl.col.date.dt.month())
    .unnest("variable")
    .filter(shapeType="ADM0")
)

In [7]:
# Cubic spline from Blue Marble Next Gen
cs = CubicSpline([0,0.25,0.75,1], [0, 179, 240, 255])

In [8]:
means = long.group_by(["shapeGroup", "shapeName", "band", "landCover", "month"]).agg(pl.col.meanReflectance.mean()).drop_nans().with_columns(
                hex=pl.col.meanReflectance
                    .map_elements(lambda meanRefl: float(cs((meanRefl * 0.0001))), return_dtype=pl.Float32)
                    .cast(pl.UInt8, strict=False)
            ).drop_nulls()
means

shapeGroup,shapeName,band,landCover,month,meanReflectance,hex
str,str,str,str,i8,f64,u8
"""LUX""","""Luxembourg""","""green""","""builtup""",10,628.683025,60
"""USA""","""United States""","""red""","""builtup""",4,1074.39096,96
"""LUX""","""Luxembourg""","""green""","""savanna""",4,605.952752,58
"""HRV""","""Croatia""","""blue""","""wetland""",7,307.365159,30
"""PNG""","""Papua New Guinea""","""blue""","""savanna""",1,219.604045,22
…,…,…,…,…,…,…
"""UZB""","""Uzbekistan""","""red""","""shrubs""",10,2343.713745,172
"""COG""","""Congo, Rep of the""","""red""","""builtup""",4,618.790578,59
"""COL""","""Colombia""","""blue""","""crops""",12,330.350788,32


In [9]:
def to_rgb(dataframe, groupby):
    groupby = set(groupby)
    later_select = groupby-{"band"}
    means = dataframe.group_by(groupby).agg(pl.col.meanReflectance.mean()).drop_nans()
    return (
        means
            .with_columns(
                hex=pl.col.meanReflectance
                    .map_elements(lambda meanRefl: float(cs((meanRefl * 0.0001))), return_dtype=pl.Float32)
                    .cast(pl.UInt8, strict=False)
            )
            .drop_nulls()
            .pivot(index=later_select, values="hex", on="band").select(*later_select, "red", "green", "blue")
    )

In [10]:
total_mean = to_rgb(long, ["shapeGroup", "shapeName", "band", "landCover"])
mean_per_month = to_rgb(long, ["shapeGroup", "shapeName", "band", "landCover", "month"])

In [30]:
countries = {}
for groupby, data in mean_per_month.sort("landCover").group_by("shapeName", "month"):
    country_name = groupby[0]  
    month = groupby[1]
    if country_name in countries:
        countries[country_name].update({
            month: [
                {
                    "landCover": row[-4],
                    "rgb": ",".join(map(str, row[-3:]))
                }
                for row in data.iter_rows()
            ]
        })
    else:
        countries[country_name] = {
            month: [
                {
                    "landCover": row[-4],
                    "rgb": ",".join(map(str, row[-3:]))
                }
                for row in data.iter_rows()
            ]
        }
countries = {k: dict(sorted(v.items())) for k,v in countries.items()}
    

In [31]:
countries

{'Egypt': {1: [{'landCover': 'builtup', 'rgb': '106,95,61'},
   {'landCover': 'crops', 'rgb': '60,71,37'},
   {'landCover': 'forest', 'rgb': '31,39,23'},
   {'landCover': 'grassland', 'rgb': '140,119,72'},
   {'landCover': 'savanna', 'rgb': '78,74,43'},
   {'landCover': 'shrubs', 'rgb': '153,125,78'},
   {'landCover': 'water', 'rgb': '42,50,32'},
   {'landCover': 'wetland', 'rgb': '52,52,31'}],
  2: [{'landCover': 'builtup', 'rgb': '113,101,65'},
   {'landCover': 'crops', 'rgb': '64,73,39'},
   {'landCover': 'forest', 'rgb': '41,49,30'},
   {'landCover': 'grassland', 'rgb': '145,124,76'},
   {'landCover': 'savanna', 'rgb': '85,80,48'},
   {'landCover': 'shrubs', 'rgb': '159,131,82'},
   {'landCover': 'water', 'rgb': '46,54,36'},
   {'landCover': 'wetland', 'rgb': '56,56,34'}],
  3: [{'landCover': 'builtup', 'rgb': '123,109,71'},
   {'landCover': 'crops', 'rgb': '72,77,43'},
   {'landCover': 'forest', 'rgb': '42,52,30'},
   {'landCover': 'grassland', 'rgb': '153,130,81'},
   {'landCover

In [32]:
reorganized_data = {}
for country, months in countries.items():
    reorganized_data[country] = {}
    for month, land_covers in months.items():
        for entry in land_covers:
            land_cover = entry['landCover']
            rgb = entry['rgb']
            if land_cover not in reorganized_data[country]:
                reorganized_data[country][land_cover] = {}
            reorganized_data[country][land_cover][month] = rgb

In [33]:
reorganized_data

{'Egypt': {'builtup': {1: '106,95,61',
   2: '113,101,65',
   3: '123,109,71',
   4: '137,119,80',
   5: '147,127,86',
   6: '149,129,87',
   7: '143,125,84',
   8: '136,119,79',
   9: '133,114,76',
   10: '126,108,72',
   11: '114,99,64',
   12: '107,95,61'},
  'crops': {1: '60,71,37',
   2: '64,73,39',
   3: '72,77,43',
   4: '96,91,55',
   5: '115,101,64',
   6: '108,97,60',
   7: '89,88,51',
   8: '82,82,47',
   9: '94,87,53',
   10: '93,84,51',
   11: '75,75,42',
   12: '65,72,38'},
  'forest': {1: '31,39,23',
   2: '41,49,30',
   3: '42,52,30',
   4: '52,63,36',
   5: '54,64,35',
   6: '50,62,33',
   7: '49,61,32',
   8: '44,56,32',
   9: '42,52,31',
   10: '39,46,25',
   11: '29,37,21',
   12: '27,35,19'},
  'grassland': {1: '140,119,72',
   2: '145,124,76',
   3: '153,130,81',
   4: '166,141,90',
   5: '176,148,97',
   6: '178,150,98',
   7: '174,148,95',
   8: '169,142,91',
   9: '165,138,89',
   10: '159,132,84',
   11: '149,125,78',
   12: '142,120,73'},
  'savanna': {1: '78

In [ ]:
html_template = """<!DOCTYPE html>
<html>
<head>
    <title>Land Use Color Palettes</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 20px;
            padding: 0;
        }
        h1, h2, h3 {
            margin-top: 20px;
        }
        .palette {
            display: flex;
            flex-wrap: wrap;
            margin-bottom: px;
        }
        .color-block {
            width: 60px;
            height: 30px;
            margin: 0px;
            text-align: center;
            font-size: 10px;
            color: #000;
            display: flex;
            align-items: center;
            justify-content: center;
        }
.landuse {
    width: 100px;
    text-align: center;
    color: #000;
    display: flex;
    align-items: center;
    justify-content: right;
    margin-right: 10px;
}
    </style>
</head>
<body>
    <h1>Land Use Color Palettes</h1>
"""

def generate_palette_html(data):
    content = ""
    for country, land_uses in data.items():
        content += f"<h2>{country}</h2>\n"
        for land_use, months in land_uses.items():
            content += "<div class='palette'>\n"
            content += f"<div class='landuse'>{land_use.capitalize()}</div>\n"
            for month, rgb in months.items():
                rgb_tuple = tuple(map(int, rgb.split(',')))
                text_color = "#000" if sum(rgb_tuple) > 382 else "#fff"  # Choose text color based on brightness
                content += (f"<div class='color-block' style='background-color: rgb({rgb}); color: {text_color};'>"
                            f"</div>\n")
            content += "</div>\n"
    return content

html_content = generate_palette_html(reorganized_data)
final_html = html_template+html_content+"</body></html>"

# Save the HTML to a file
output_file = "land_use_palettes.html"
with open(output_file, "w", encoding="utf-8") as f:
    f.write(final_html)

print(f"HTML file generated")


HTML file generated


In [44]:
# HTML Template for the palette page
html_template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Country Color Palettes</title>
    <style>

        body {
            font-family: 'Arial', sans-serif;
            margin: 0;
            padding: 0;
            background-color: #f5f5f5;
            color: #333;
        }
        header {
            background-color: #007acc;
            color: white;
            padding: 20px;
            text-align: center;
            box-shadow: 0 2px 5px rgba(0, 0, 0, 0.1);
        }
        h1 {
            margin: 0;
            font-size: 2.5em;
        }
        .container {
            max-width: 1200px;
            margin: 20px auto;
            padding: 20px;
            background: white;
            border-radius: 8px;
            box-shadow: 0 2px 10px rgba(0, 0, 0, 0.1);
        }
        .country {
            margin-bottom: 30px;
            border-bottom: 1px solid #e0e0e0;
            padding-bottom: 15px;
        }
        .country h2 {
            font-size: 1.8em;
            margin-bottom: 10px;
            color: #007acc;
            border-left: 4px solid #007acc;
            padding-left: 10px;
        }
        .month {
            display: flex;
            align-items: center;
            margin-bottom: 0px;
        }
        .month-label {
            font-size: 1.2em;
            font-weight: bold;
            margin-right: 15px;
            width: 100px;
            color: #555;
        }
        .palette {
            display: flex;
            flex-wrap: wrap;
            gap: 8px;
        }
        .color {
            width: 100px;
            height: 50px;
            display: flex;
            align-items: center;
            justify-content: center;
            font-size: 0.9em;
            font-weight: bold;
            text-transform: capitalize;
            color: white;
            #border-radius: 8px;
            box-shadow: 0 2px 5px rgba(0, 0, 0, 0.15);
            transition: transform 0.2s ease-in-out;
        }
        .color:hover {
            transform: scale(1.1);
        }
        footer {
            text-align: center;
            padding: 20px;
            margin-top: 40px;
            background-color: #007acc;
            color: white;
            font-size: 0.9em;
        }
        .landcover-title {
            font-size: 1em;
            font-weight: bold;
            color: #666;
            margin-bottom: 5px;
        }
        .titles {
           display: flex;
        }
    </style>
</head>
<body>
    <header>
        <h1>Country Color Palettes</h1>
    </header>
    <div class="container">
        {% for country, landuse in countries.items() %}
        <div class="country">
            <h2>{{ country }}</h2>
            {% for landuse, months in landuse.items() %}
                <div class="month-label">{{ landuse }}</div>
                <div class="palette">
                {% for month, rgb in months.items() %}
                    <div class="color" style="background-color: rgb({{ rgb }});"></div>
                {% endfor %}
            {% endfor %}
            </div>
            </div>
            {% endfor %}
    </div>
    <footer>
        &copy; 2025 Country Color Palettes. All Rights Reserved.
    </footer>
</body>
</html>
"""

# Render the HTML
template = Template(html_template)
html_content = template.render(countries=reorganized_data)

# Save the HTML to a file
output_file = "land_cover_palettes.html"
with open(output_file, "w") as f:
    f.write(html_content)

print(f"HTML file has been generated and saved as {output_file}.")


HTML file has been generated and saved as land_cover_palettes.html.
